In [28]:
import os
import glob
import sys
import argparse
import json
import errno
import gensim
from nltk import *
from gensim import corpora, models
# files = glob.glob(p + "/data/1.txt")

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string


import re
tokenizer = RegexpTokenizer(r'\w+')

p=os.getcwd()
stopwords = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatize = WordNetLemmatizer()

def cleaning_text(text):
    letters_only = re.sub("[^a-zA-Z]", " ", text)
    words = tokenizer.tokenize(letters_only.lower())
    stops = set(nltk.corpus.stopwords.words("english"))
    _words = [w for w in words if not w in stopwords]
    return _words

def cleaning(article):
    zero = re.sub("[^a-zA-Z]", " ", article)
    one = " ".join([i for i in zero.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three

cleaned_text = []
def loadtext(testfiles):
     for name in testfiles:
        try:
            with open(name, 'r',encoding='utf-8') as f:
             for line in f:
                if line.startswith("[OCR_aligned]"):
                    raw_corpus = line[len("[OCR_aligned]") + 1:]
                    # raw_corpus = cleaning(raw_corpus)
                    cleaned_text.append(raw_corpus)
        except IOError as exc: #Not sure what error this is
            if exc.errno != errno.EISDIR:
                raise

In [29]:
def main():
    parser = argparse.ArgumentParser(description="Historical OCRed Topic Modeling Script", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    # parser.add_argument('-i','--input', type=str, help="Input file or directory (*.txt files)", action='store',required=True)
    # args = parser.parse_args()

    if os.path.isdir(p):
        testfiles = []
        for f in glob.glob(p+ "/data/en_periodicals/*.txt"):
            testfiles.append(f)

    else:
        testfiles = [p]

    loadtext(testfiles)
    # print(cleaned_text)
   
    # text = loadtext(testfiles)
    # print(text)
main()

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import NMF, LatentDirichletAllocation

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

# dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))

documents = cleaned_text

no_features = 10000
no_topics = 20
# NMF is able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(documents)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [31]:
# Run NMF
nmf = NMF(n_components=no_topics, random_state=4, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

no_top_words = 25
print("\nTopics in LDA model:\n")
display_topics(lda, tf_feature_names, no_top_words)

print("\nTopics in NMF model:\n")
display_topics(nmf, tfidf_feature_names, no_top_words)

/home/mutuvi/miniconda3/envs/keras/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)



Topics in LDA model:

Topic 0:
mr london new street day th esq year man tho 000 time general having hand place large tlie saturday public following 10 said tbe rev
Topic 1:
mr tlie exeter street new house london day esq tbe years th railway time general road good john having best man 000 st mrs tho
Topic 2:
jan street mr th new esq london day messrs john january feb house price published hull attorney dec 6d general company public said 31 french
Topic 3:
mr street tbe tlie london esq public day house clock director said life john new st mrs morning january given time sir country magistrate fine
Topic 4:
mr tbe th day esq street royal london place time new general st 000 clock order john house king exeter troops iu men said william
Topic 5:
mr tbe street day time new house london said th tho tlie place having man 000 st esq year good public john sir present aud
Topic 6:
mr esq railway london tbe director day time sir street house efq william lord john new having general years right aud